In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import gc
import os
import re
import sklearn
import sys

from joblib import Parallel, delayed
from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../src')
sys.path.append(src_dir)

In [3]:
%aimport data.movielens_imdb

from data.movielens_imdb import load_into_dataframe

In [4]:
ML_ROOT = "/media/felipe/SAMSUNG/movielens/ml-10m/ml-10M100K/"
IMDB_ROOT = "/media/felipe/SAMSUNG/imdb/"

PATH_TO_MOVIES = ML_ROOT + "/movies.dat"
PATH_TO_TAG_ASSIGNMENTS = ML_ROOT + "/tags.dat"
# slightly modified version, without the top 15 lines
PATH_TO_MOVIE_PLOTS = IMDB_ROOT+"/plot.list.tail"



In [5]:
docs_df = load_into_dataframe(PATH_TO_MOVIES,PATH_TO_TAG_ASSIGNMENTS,PATH_TO_MOVIE_PLOTS)

In [6]:
docs_df.head(10)

,movie_id,title,genres,tags,plot
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"lots of heart,the boys,kids movie,light,avi,Jo...",A little boy named Andy loves to be in his ro...
1,2,Jumanji (1995),Adventure|Children|Fantasy,"Robin Williams,Children,adapted from:book,chil...",After being trapped in a jungle board game fo...
2,3,Grumpier Old Men (1995),Comedy|Romance,"old,moldy,comedinha de velhinhos engraÃƒÂ§ada,...",Things don't seem to change much in Wabasha C...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,girl movie,This story based on the best selling novel by...
4,5,Father of the Bride Part II (1995),Comedy,"Fantasy,sequel,steve martin,worst movies ever,...","In this sequel to ""Father of the Bride"", Geor..."
5,6,Heat (1995),Action|Crime|Thriller,"In The Nucleus,too long,R,1,Top 10 ever,Want,C...",Hunters and their prey--Neil and his professi...
6,7,Sabrina (1995),Comedy|Romance,"based on a play,remadeAs:Sabrina(1995),remakeO...","While she was growing up, Sabrina Fairchild s..."
7,8,Tom and Huck (1995),Adventure|Children,"author:Mark Twain,adapted from:book","A mischievous young boy, Tom Sawyer ('Jonatha..."
8,9,Sudden Death (1995),Action,"Can't remember,Jean-Claude Van Damme",Some terrorists kidnap the Vice President of ...
9,10,GoldenEye (1995),Action|Adventure|Thriller,"murder,Best Of the Brosnon Bonds,sequel,violen...",When a deadly satellite weapon system falls i...


In [8]:
docs_df.describe(include=['object'])

,title,genres,tags,plot
count,6006,6006,4429,6006
unique,5977,588,3965,6004
top,War of the Worlds (2005),Drama,less than 300 ratings,Ray Ferrier (Cruise) is a divorced dockworker...
freq,4,894,91,2


In [10]:
docs_df.describe(exclude=['object'])

,movie_id
count,6006.000000
mean,12621.118715
std,17630.438571
min,1.000000
25%,2549.500000
50%,5116.500000
75%,8490.000000
max,65133.000000
